In [2]:
with open('paloozabot.log','r') as file:
    log_data = file.read()


In [3]:
from utils.listify_logs import listify_log
from utils.jsonify_log import jsonify_log
log_list = listify_log(log_data)
test_json = jsonify_log(log_list[0]) # test

# Datetime TIMESTAMP,
# Level VARCHAR(8),
# Module TEXT,
# Context TEXT,
# Message TEXT

In [5]:
(203*3)/60

10.15

In [8]:
processed_logs = []
for index, log in enumerate(log_list):
    print(f"working on log {index}\n")
    print(log)
    print("\n")
    processed_log = jsonify_log(log)
    processed_logs.append(processed_log)
    print(f"\nprocessed log {index}\n")



working on log 0

05/01/2024 03:33:36 PM:WARNING:discord.client:client:295:__init__: PyNaCl is not installed, voice will NOT be supported




processed log 0

working on log 1

05/01/2024 03:33:36 PM:DEBUG:discord.client:client:1989:event: on_ready has successfully been registered as an event




processed log 1

working on log 2

05/01/2024 03:33:36 PM:DEBUG:asyncio:selector_events:64:__init__: Using selector: EpollSelector




processed log 2

working on log 3

05/01/2024 03:33:36 PM:INFO:discord.client:client:603:login: logging in using static token




processed log 3

working on log 4

05/01/2024 03:33:37 PM:INFO:discord.gateway:gateway:561:received_message: Shard ID None has connected to Gateway (Session ID: 8d0f6a8f9f354eda4c4327c900d5c823).




processed log 4

working on log 5

05/01/2024 03:33:39 PM:INFO:discord:bot:30:on_ready: Logged in as paloozabot




processed log 5

working on log 6

05/01/2024 03:34:40 PM:INFO:discord:bot:35:hello: a_na4603 invoked hello command




p

In [ ]:
# options
# 
# discord:bot <- message 
# discord.gateway <- system
# invoked <- command
# 1. go and edit the prompt
# 2. write some logic to deal with it
# we will have reference for what keys we expect
# if a key that we expect is not in there
# we'll add it, along with a missing value place holder
# if there's a key that SHOULD NOT be in there, then remove it and it's value 



In [25]:
log_type_totals = {"message":0,"system":0,"command":0,"none":0}

for log in log_list:
    if "invoked" in log:
        log_type_totals["command"] +=1
    elif "on_message" in log:
        log_type_totals["message"] +=1
    elif "discord.gateway" in log or "discord.client" in log or "discord.ext.commands" in log: 
        log_type_totals["system"] +=1
    else:
        log_type_totals["none"] += 1
        print(log)




05/01/2024 03:33:36 PM:DEBUG:asyncio:selector_events:64:__init__: Using selector: EpollSelector

05/01/2024 03:33:39 PM:INFO:discord:bot:30:on_ready: Logged in as paloozabot

05/01/2024 08:54:10 PM:DEBUG:asyncio:selector_events:64:__init__: Using selector: EpollSelector

05/01/2024 08:54:13 PM:INFO:discord:bot:34:on_ready: logged in as paloozabot

05/01/2024 10:13:40 PM:DEBUG:asyncio:selector_events:64:__init__: Using selector: EpollSelector

05/01/2024 10:13:43 PM:INFO:discord:bot:42:on_ready: logged in as paloozabot

05/01/2024 10:54:53 PM:DEBUG:asyncio:selector_events:64:__init__: Using selector: EpollSelector

05/01/2024 10:54:55 PM:INFO:discord:bot:42:on_ready: logged in as paloozabot

05/01/2024 11:34:14 PM:INFO:discord:bot:67:on_reaction_add: a_na4603 added 👋 to message ID 1235373520414834790: 

05/02/2024 02:26:06 PM:INFO:discord:bot:67:on_reaction_add: a_na4603 added 🫡 to message ID 1235388025698258965: 

05/02/2024 02:26:37 PM:INFO:discord:bot:67:on_reaction_add: a_na4603 add

In [ ]:
# observations
# we were ignoring reactions - probably a bad idea
# we should define a table for them
# but if we do this, we also need to change our message table to record message id

In [27]:
import pickle

with open('processed_logs.pkl','wb') as f:

    pickle.dump(processed_log,f)

In [24]:
log_type_totals

{'message': 51, 'system': 131, 'command': 5, 'none': 16}

In [13]:
print(processed_logs[200])
print(processed_logs[141])

{'db_name': 'System', 'data': {'Datetime': '2024-05-08T19:00:53', 'Level': 'INFO', 'Module': 'discord.gateway', 'Context': 'gateway:566:received_message', 'Message': 'Shard ID None has successfully RESUMED session 47da0e432872b0d38baa5acf8fee8604'}}
{'db_name': 'Message', 'data': {'Message_time': '2024-05-04T19:10:40', 'User_id': 'a_na4603', 'Channel_name': '#project', 'Channel_id': 1225875744891207761, 'Message': '@everyone update:, Message ID: 1236394587451293836', 'Mentions': ['@everyone'], 'Emojis': []}}


In [11]:
errors = 0
for log in processed_logs:
    if log['db_name'] == 'error':
        errors += 1

errors/len(processed_logs)

0.034482758620689655

In [13]:
# tag outgoing jsons as belonging to either the message, system, command


# send it to the postgres database
import psycopg2
import json

# Establish a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="palooza",
    user="postgres",
    password=os.environ["dbpassword"]
)

# Create a cursor object
cur = conn.cursor()

# Define your records
records = [
    {"name": "John", "age": 25, "city": "New York"},
    {"name": "Alice", "age": 30, "city": "London"},
    {"name": "Bob", "age": 35, "city": "Paris"}
]

# Iterate over the records and insert them into the database
for record in records:
    name = record["name"]
    age = record["age"]
    city = record["city"]
    
    query = "INSERT INTO your_table (name, age, city) VALUES (%s, %s, %s)"
    values = (name, age, city)
    
    cur.execute(query, values)

# Commit the changes and close the cursor and connection
conn.commit()
cur.close()
conn.close()


In [ ]:
# start with a log then get a json
# we turn it into a single tabular record then we transfer it to the database
# the AI is giving us different jsons (3 different kinds)
# it spits out a json and we need to know what table it belongs to.
# we'll use the length of the dict for this rather than asking gpt to figure it out